In [1]:
from langchain.document_loaders import WebBaseLoader

In [50]:
loader = WebBaseLoader("https://hatkestory.com/about-us/")
data = loader.load()

In [51]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 100,
    length_function = len,
    add_start_index = True,)
texts = text_splitter.split_documents(data)

In [37]:
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

In [1]:
import os
openai_api_key = 'key'
pinecone_api_key = 'pineconekey'
os.environ['PINECONE_API_KEY'] = pinecone_api_key
os.environ['OPENAI_API_KEY'] = openai_api_key

In [38]:
embeddings = OpenAIEmbeddings(openai_api_key=os.environ['OPENAI_API_KEY'])

In [41]:
pc= pinecone.Pinecone(
            api_key= os.environ['PINECONE_API_KEY'],
            environment= 'us-east-1'
)
index_name = pc.Index('index-1')

In [43]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

In [52]:
vectordb = Pinecone.from_documents(texts, embeddings, index_name='index-1')

In [53]:
retriever = vectordb.as_retriever()

In [54]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

memory = ConversationBufferMemory(memory_key="chat_history", return_messages= True)
chain = ConversationalRetrievalChain.from_llm(llm, retriever= retriever, memory= memory)

In [55]:
query = 'where did gopi met rohit first?'
chain.invoke({'question': query})

{'question': 'where did gopi met rohit first?',
 'chat_history': [HumanMessage(content='where did gopi met rohit first?'),
  AIMessage(content='Gopi met Rohit for the first time at the JNTU campus in Hyderabad.')],
 'answer': 'Gopi met Rohit for the first time at the JNTU campus in Hyderabad.'}

In [56]:
query = 'what is the initial conversation happened between gopi and rohit?'
chain.invoke({'question': query})

{'question': 'what is the initial conversation happened between gopi and rohit?',
 'chat_history': [HumanMessage(content='where did gopi met rohit first?'),
  AIMessage(content='Gopi met Rohit for the first time at the JNTU campus in Hyderabad.'),
  HumanMessage(content='what is the initial conversation happened between gopi and rohit?'),
  AIMessage(content='The initial conversation between Gopi and Rohit was:\n“Hey, hi, I am Rohit. Where are you from?” \n“Hello, Rohit. I am Gopi. I am from Vijayawada.”')],
 'answer': 'The initial conversation between Gopi and Rohit was:\n“Hey, hi, I am Rohit. Where are you from?” \n“Hello, Rohit. I am Gopi. I am from Vijayawada.”'}

In [57]:
query = 'when did gopi and rohit decided to start the platform?'
chain.invoke({'question': query})

{'question': 'when did gopi and rohit decided to start the platform?',
 'chat_history': [HumanMessage(content='where did gopi met rohit first?'),
  AIMessage(content='Gopi met Rohit for the first time at the JNTU campus in Hyderabad.'),
  HumanMessage(content='what is the initial conversation happened between gopi and rohit?'),
  AIMessage(content='The initial conversation between Gopi and Rohit was:\n“Hey, hi, I am Rohit. Where are you from?” \n“Hello, Rohit. I am Gopi. I am from Vijayawada.”'),
  HumanMessage(content='when did gopi and rohit decided to start the platform?'),
  AIMessage(content='Gopi and Rohit decided to start the platform, HATKESTORY.COM, in 2016.')],
 'answer': 'Gopi and Rohit decided to start the platform, HATKESTORY.COM, in 2016.'}

In [58]:
query = 'what is the motivation for rohit and gopi to start hatkestory?'
chain.invoke({'question': query})

{'question': 'what is the motivation for rohit and gopi to start hatkestory?',
 'chat_history': [HumanMessage(content='where did gopi met rohit first?'),
  AIMessage(content='Gopi met Rohit for the first time at the JNTU campus in Hyderabad.'),
  HumanMessage(content='what is the initial conversation happened between gopi and rohit?'),
  AIMessage(content='The initial conversation between Gopi and Rohit was:\n“Hey, hi, I am Rohit. Where are you from?” \n“Hello, Rohit. I am Gopi. I am from Vijayawada.”'),
  HumanMessage(content='when did gopi and rohit decided to start the platform?'),
  AIMessage(content='Gopi and Rohit decided to start the platform, HATKESTORY.COM, in 2016.'),
  HumanMessage(content='what is the motivation for rohit and gopi to start hatkestory?'),
  AIMessage(content="Rohit and Gopi were motivated to start Hatkestory because they wanted to share the stories of ordinary people living extraordinary lives. They believed that these stories deserved more attention and cel